# **Dependências**

In [ ]:
!pip install pypdf

In [ ]:
# !pip freeze > "/content/drive/MyDrive/Melina - Aex/py_3.11.13_packages.txt"

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import re

from tqdm import tqdm
from pypdf import PdfReader

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# **Lendo os dados**

In [ ]:
def pdf_to_string(folder: str) -> dict:
  """
    Transforma os arquivos .pdf em string.
    Salva em um dicionário.
  """

  files = {"doc":[],
           "String":[],
           "# Páginas":[]}

  for fname in tqdm(os.listdir(folder)):
    if not fname.lower().endswith(".pdf"):
        continue

    path = os.path.join(folder, fname)
    reader = PdfReader(path)
    content = [page.extract_text() + " " for page in reader.pages]
    files["String"].append("[[end_page]]".join(content))
    files["# Páginas"].append(len(content))
    files["doc"].append(fname.rsplit(r".pdf")[0])

  return files

In [ ]:
# docs = pdf_to_string("/content/drive/MyDrive/Fichas AEX")
# pd.DataFrame(docs).to_parquet("/content/drive/MyDrive/Melina - Aex/aex_strings.parquet")

# **EDA**

In [ ]:
df = pd.read_parquet("/content/drive/MyDrive/Melina - Aex/aex_strings.parquet")
df.head()

,doc,String,# Páginas
0,aexGerarPDF_AEX-FMRP-00020.01,Pró-Reitoria de\n Cultura e Extensão Universi...,2
1,aexGerarPDF_AEX-ICMC-00032.01,Pró-Reitoria de\n Cultura e Extensão Universi...,2
2,aexGerarPDF_AEX-FZEA-00060.01,Pró-Reitoria de\n Cultura e Extensão Universi...,2
3,aexGerarPDF_AEX-FOB-00004.01,Pró-Reitoria de\n Cultura e Extensão Universi...,6
4,aexGerarPDF_AEX-FOB-00022.01,Pró-Reitoria de\n Cultura e Extensão Universi...,4


In [ ]:
# Todos os arquivos começam com Pró-Reitoria de Cultura e Extensão Universitária?
df.String.str.startswith(" Pró-Reitoria de\n Cultura e Extensão Universitária").all()

np.True_

In [ ]:
# Todos as páginas começam com Pró-Reitoria de Cultura e Extensão Universitária?

df[df["# Páginas"] > 1].String.str.contains("\[\[end_page\]\] Pró-Reitoria de\n Cultura e Extensão Universitária").all()

np.True_

In [ ]:
# Qual é o número médio de páginas?
print(round(df["# Páginas"].mean(), 1))
print(round(df["# Páginas"].std(), 1))

3.1
1.5


# **Limpeza e Tratamemnto**

In [ ]:
# Removendo cabeçalho
df["String"] = df.String.str.replace(" Pró-Reitoria de\n Cultura e Extensão Universitária\n", " ")
df["String"] = df.String.str.replace("\[\[end_page\]\] Pró-Reitoria de\n Cultura e Extensão Universitária\n", " ")
df["String"] = df.String.str.replace(r"\s*\[\[end_page\]\]\s*", "\n", regex=True)

# Remove quebra de linhas
df["String"] = df["String"].str.replace(r"\n(\w+)", r"\n \1", regex=True)
df["String"] = df["String"].str.replace("\n", "")

# Remove sequências de espaços em branco
df["String"] = df["String"].str.replace("\s+", " ", regex=True)

# Remove marcação de página
df["String"] = df["String"].str.replace("P[áa]gina\s\d{1,}\s*de\s*\d{1,}.*?\d{1,2}\/\d{1,2}\/\d{2,4}", " ", regex=True)

# Lida com ".palavra" -> ". palavra"
df["String"] = df["String"].str.replace(r"\.(\w+)", r". \1", regex=True)

# **Estruturando os dados**

In [ ]:
df.head()

,doc,String,# Páginas
0,aexGerarPDF_AEX-FMRP-00020.01,Docente Responsável: 3223241 - Regina Yoneko ...,2
1,aexGerarPDF_AEX-ICMC-00032.01,Docente Responsável: 4831147 - Eduardo do Val...,2
2,aexGerarPDF_AEX-FZEA-00060.01,Docente Responsável: 5412221 - Caio Filipe da...,2
3,aexGerarPDF_AEX-FOB-00004.01,Docente Responsável: 1851311 - Regina Tangeri...,6
4,aexGerarPDF_AEX-FOB-00022.01,Docente Responsável: 1851311 - Regina Tangeri...,4


In [ ]:
# Supõe só um docente responsável por documento
# Supõe que o comprimento de NUSP é maior do que 3 e menor do que 9 caracteres

df["Docente Responsável - NUSP"] = df["String"].str.extract(r"(\d{3,9})\s*-\s*")
df["Docente Responsável - Nome"] = df["String"].str.extract(r"\d{3,9}\s*-\s*(.*?)\s+[Tt][íi]tulo")

In [ ]:
# Todas as linhas possuem nome
df[df["Docente Responsável - Nome"].isna()]

,doc,String,# Páginas,Docente Responsável - NUSP,Docente Responsável - Nome


In [ ]:
# Todas as linhas possuem NUSP
df[df["Docente Responsável - NUSP"].isna()]

,doc,String,# Páginas,Docente Responsável - NUSP,Docente Responsável - Nome


In [ ]:
# Supõe que o próximo item do documento começa com "Atividade AEX"
df["Título da Atividade"] = df["String"].str.extract(r"[Tt][íi]tulo da [Aa]tividade:?\s(.*?)\.?\s?Atividade\sAEX")

In [ ]:
df["AEX ID"] = df["String"].str.extract(r"Atividade AEX\-?(.*?)\sUnidade\/Colegiado")

In [ ]:
# Supõe que todas as unidades possuem sigla
# Supõe que as siglas têm entre 2 e 15 caracteres
df["Unidade Nome"] = df["String"].str.extract(r"Unidade\/Colegiado:(.*?)\(.{2,15}\)")
df["Unidade Sigla"] = df["String"].str.extract(r"Unidade\/Colegiado:.*?\((.{2,15})\)")

In [ ]:
df["Descrição da Atividade"] = df["String"].str.extract(r"Unidade\/Colegiado:.*?\(.{2,9}\)\s?(.*)[Dd]escrição da [Aa]tividade:?")

In [ ]:
# Supõe que o grupo social alvo da atividade vem antes da carga horária da atividade
df["Grupo Social Alvo"] = df["String"].str.extract(r"[Gg]rupo [Ss]ocial [Aa]lvo da [Aa]tividade:?(.*)[Ca]arga [Hh]or[aá]ria da [Aa]tividade")

In [ ]:
# Supõe que o grupo social alvo da atividade vem antes da carga horária da atividade
df["Grupo Social Alvo"] = df["String"].str.extract(r"[Gg]rupo [Ss]ocial [Aa]lvo da [Aa]tividade:?(.*)[Ca]arga [Hh]or[aá]ria da [Aa]tividade")

In [ ]:
# Supõe que o horário tem no máximo 4 dígitos para o horário e 2 dígitos para os minutos
df["Carga Horária da Atividade (HH:mm)"] = df["String"].str.extract(r"[Ca]arga [Hh]or[aá]ria da [Aa]tividade:?.*?(\d{1,4}:\d{1,2})")
df["Carga Horária do Docente Responsável (HH:mm)"] = df["String"].str.extract(r"[Ca]arga [Hh]or[aá]ria do [Dd]ocente\s[Rr]esponsável:?.*?(\d{1,4}:\d{1,2})")

In [ ]:
# Supõe que a informação é imediatamente anterior a Indicadores de avaliação da atividade
df["Objetivos, metas e resultados esperados"] = df["String"].str.extract(r"[Oo]bjetivos, [Mm]etas e [Rr]esultados esperados:?(.*?)[Ii]ndicadores de [Aa]valia[çc][ãa]o da [Aa]tividade")

In [ ]:
# Supõe que precede indicadores de avaliação dos alunos
df["Indicadores de Avaliação da Atividade"] = df["String"].str.extract("[Ii]ndicadores de [Aa]valiação da [Aa]tividade:?(.*?)[Ii]ndicadores de [Aa]valiação dos [Aa]lunos USP:?")

In [ ]:
# Precede "Pré-requisitos" ou "Adequação à estratégia ODS"
df["Indicadores de Avaliação dos Alunos"] = df["String"].str.extract("[Ii]ndicadores de [Aa]valiação dos [Aa]lunos USP:?(.*?)(?:[Pp]ré-?[Rr]equisito|[Aa]dequa[çc][ãa]o [àa] [Ee]strat[ée]gia ODS)")

In [ ]:
# Supõe que vem antes de Metodologia, metas, ações e resultados esperados com os objetivos ODS indicados
df[["Adequação à estratégias ODS"]] =  df["String"].str.extract("[Aa]dequa[çc][ãa]o [àa] [Ee]strat[ée]gia ODS:?(.*?)Metodologia, metas, ações e resultados esperados com os objetivos ODS indicados:?")

# Supõe que vem antes da bibliografia ou oferecimento
df[["MMAR com os objetivos ODS indicados"]] =  df["String"].str.extract("Metodologia, metas, ações e resultados esperados com os objetivos ODS indicados:?(.*?)(?:[Bb]ibliografia|Oferecimento\(s\)\:)")

## **Corrigindo erros**

In [ ]:
# Corresponsáveis pela atividade não foi extraido -> Tabela com pouca consistência de documento para documento
df[["String"]].head()

,String
0,Docente Responsável: 3223241 - Regina Yoneko ...
1,Docente Responsável: 4831147 - Eduardo do Val...
2,Docente Responsável: 5412221 - Caio Filipe da...
3,Docente Responsável: 1851311 - Regina Tangeri...
4,Docente Responsável: 1851311 - Regina Tangeri...


In [ ]:
df.loc[541, "Descrição da Atividade"] = """Esta atividade tem como finalidade engajar alunos(as) de graduação da USP no Programa de Visitas Monitoradas do CEBIMar para atuarem como monitores(as), abordando junto ao público visitante temas relacionados à divulgação das ciências marinhas e educação ambiental. Tal tarefa é concentrada nos meses de recesso escolar (entre dezembro e fevereiro e em julho), quando os(as) alunos(as) têm disponibilidade de vir ao CEBIMar – localizado na cidade de São Sebastião – e existe alta demanda devido à temporada turística da cidade. Neste período, eles(as) participam também do processo de agendamento das visitas e, com o apoio de técnicos especializados do CEBIMar, podem auxiliar nas coletas e manutenção de organismos. Passam ainda por um treinamento teórico-prático que os(as) capacita para versarem sobre os temas abordados durante a monitoria: aspectos físico- químicos do ambiente marinho; ecossistemas marinhos; comunidades marinhas planctônica, bentônica e nectônica; sistemática filogenética dos seres vivos; anátomo-fisiologia de organismos marinhos costeiros; impactos naturais e antrópicos no ambiente marinho; conceitos de educação ambiental e divulgação científica; noções da prática de mergulho livre. Durante o período letivo, os(as) alunos(as) monitoram turmas apenas quando o calendário acadêmico da USP não prevê aulas curriculares (usualmente durante as semanas Santa e da Pátria). Também ao longo desta temporada, realizam trabalhos à distância, como leitura e discussão de textos relacionados ao contexto técnico e pedagógico da monitoria, além de tabulação e análise de formulários de avaliação aplicados ao final das visitas monitoradas. Descrição da atividade: Grupo social alvo da atividade: As visitas monitoradas são destinadas a turmas de professores e estudantes, desde a pré-escola até o nível universitário, passando pelo ensino técnico e pela educação especial, como APAE e terceira idade. Outros grupos organizados, como turistas, veranistas e a população residente, são também frequentes participantes, totalizando em torno de 1200 visitantes ao ano, a maior parte proveniente dos municípios do Litoral Norte de São Paulo e da capital paulista, mas também de outros estados e países. Carga horária da atividade: 480:00 Carga horária do docente responsável: 4:00 Carga horária previstaCorresponsáveis pela atividade Vínculo Atuação Luciano Douglas dos Santos Abel 240:00Servidor (CEBIMar) Corresponsável Waldir Jose de Paula 60:00Servidor (CEBIMar) Corresponsável Objetivos, metas e resultados esperados: O intuito principal do oferecimento de visitas monitoradas ao CEBIMar/USP é difundir o conhecimento de modo a aprimorar o senso crítico sobre questões científicas e ambientais da população. Ainda, pretende-se que os visitantes reconheçam a importância da investigação científica, que reflitam sobre o equilíbrio econômico, social e ambiental frente às atividades humanas no ambiente marinho, aprimorando valores e comportamentos em relação ao ambiente, e que ampliem o conhecimento sobre a biodiversidade marinha. Especificamente quanto aos(as) alunos(as), objetivam-se: a qualificação científica para versarem sobre aspectos de interesse acadêmico, socioambiental e econômico; a elevação da capacidade de comunicação; a conscientização sobre a responsabilidade da universidade quanto à geração e transferência de conhecimento, bem como às aplicações tecnológicas. Indicadores de avaliação da atividade: O acompanhamento da atividade se dará com a aplicação de formulários de avaliação aos visitantes quanto à logística do procedimento como do conhecimento adquirido. Ainda, será também incentivado a preparação de material para divulgação científica nas mídias virtuais do CEBIMar, sendo que os indicadores de avaliação gravitarão no alcance, engajamento, taxa de cliques, tráfego social, taxa de rejeição, número de compartilhamentos e análise dos comentários. Essas métricas permitirão aprimorar os procedimentos de acolhimento dos visitantes bem como as publicações conseguintes a fim de, eventualmente, corrigir as estratégias de comunicação empregadas. Indicadores de avaliação dos alunos USP: Página 1 de 2Emitido em 25/07/2025. O retorno pessoal e informal que os(as) próprios(as) alunos(as) fornecerão ao longo da atividade será um indicador bastante valioso que auxiliará na planificação das ações. Neste sentido, a dinâmica, tanto das visitas monitoradas como do treinamento visando à capacitação dos(as) alunos(as), será aprimorada em função das opiniões, críticas e sugestões por eles(as) expressas. Ademais, ao final do processo da atividade que cobre um ciclo de trabalho anual, será confeccionado pelos(as) alunos(as) um relatório crítico-descritivo de todo o processo, inclusive com a análise das avaliações fornecidas pelos(as) visitantes. Pré-requisito: Não há. Adequação à estratégia ODS: Vida na Água Metodologia, metas, ações e resultados esperados com os objetivos ODS indicados As diretrizes empregadas com relação ao objetivo ODS 14, vida na água, no que se refere à metodologia, metas e ações, seguem as mesmas premissas gerais utilizadas na descrição da atividade. O mote principal é conscientizar a opinião pública sobre a importância da preservação da biodiversidade marinha e o imprescindível papel contribuidor da pesquisa científica nesse processo, inserindo, nesse procedimento, alunos(as) de graduação da USP, os(as) quais receberão treinamento abrangente para liderar visitas monitoradas ao CEBIMar, cobrindo temas científicos, socioambientais e econômicos, como poluição, biodiversidade e comércio marítimo. Excursões serão conduzidas com materiais didáticos interativos, incluindo recursos desenvolvidos pelo CEBIMar, adaptados à diversidade do público a fim de garantir uma comunicação eficaz dada a elasticidade nas faixas etárias e nos níveis educacionais e socioeconômicos dos visitantes. A experiência de liderança proporcionará aos(às) alunos(as) uma compreensão prática da divulgação científica, promovendo uma visão crítica da importância dos centros de pesquisa. Além disso, o recrutamento de graduandos(as) para atividades de monitoria aumentará o número de visitantes ao CEBIMar/USP e aprimorará a dinâmica das visitas. Dessa forma, espera-se capacitar a sociedade e futuras gerações de pesquisadores(as) e profissionais com visão crítica sobre diversos problemas ambientais que afetam a vida marinha, como os impactos locais e globais que levam à perda de biodiversidade. A expectativa é que a modificação de valores e comportamentos em relação ao ambiente e à vida marinha, bem como uma ampliação do conhecimento sobre a biodiversidade, possa criar um tecido social engajado em buscar soluções orientadas por uma gestão sustentável e pelo aumento do número de áreas marinhas protegidas."""

In [ ]:
df.loc[100, "String"]

' Docente Responsável: 6118590 - Gisela Maria Assis Título da atividade: Consulta individual a mulheres com Incontinência e Prolapsos de Órgão Pélvicos Atividade AEX-EE-00019. 01 Unidade/Colegiado: Escola de Enfermagem (EE) Consultas realizadas no ambulatório médico terapêutico Monte Azul à mulheres com Incontinência Urinária ou Prolapsos de Órgão Pélvicos. Descrição da atividade: Grupo social alvo da atividade: Moradores da Comunidade Monte Azul, moradores das áreas de abrangência das unidades de saúde do entorno, população geral que chegue por demanda aberta. Carga horária da atividade: 16:00 Carga horária do docente responsável: 400:00 Objetivos, metas e resultados esperados: Assistenciais: Objetivos: Oferecer tratamento conservador para a Incontinência Urinária e Prolapsos de Órgãos Pélvicos de forma gratuita a mulheres que procurem pelo ambulatório. Metas: Atender quatro mulheres por semana, nas sextas feiras, das 13h às 17h, totalizando 16 consultas por mês, com retornos mensais 

In [ ]:
# Linha 368 tem "x" como descrição da atividade e grupo social no documento original (outlier)
df.iloc[368]

,368
doc,aexGerarPDF_AEX-EE-00035.01
String,Docente Responsável: 2356599 - Paula Cristina...
# Páginas,2
Docente Responsável - NUSP,2356599
Docente Responsável - Nome,Paula Cristina Nogueira
Título da Atividade,"FEIRA DA SAÚDE - ""De olho no coração!"""
AEX ID,EE-00035. 01
Unidade Nome,Escola de Enfermagem
Unidade Sigla,EE
Descrição da Atividade,x


## **Finalizando**

In [ ]:
df.drop("String", axis=1, inplace=True)

df.reset_index(inplace=True, drop=True)

In [ ]:
df.rename({"doc":"Documento"}, axis=1, inplace=True)

In [ ]:
for col in df.select_dtypes(include="object").columns:
  if col not in ["Unidade Sigla", "Documento", "AEX ID", "Carga Horária da Atividade (HH:mm)", "Carga Horária do Docente Responsável (HH:mm)"]:
      df[col] = df[col].str.replace(r"\s+", " ", regex=True).str.strip()
      df[col] = df[col].apply(lambda x: x[0].upper() + x[1:] if isinstance(x, str) and x else x)

In [ ]:
df.head()

,Documento,# Páginas,Docente Responsável - NUSP,Docente Responsável - Nome,Título da Atividade,AEX ID,Unidade Nome,Unidade Sigla,Descrição da Atividade,Grupo Social Alvo,Carga Horária da Atividade (HH:mm),Carga Horária do Docente Responsável (HH:mm),"Objetivos, metas e resultados esperados",Indicadores de Avaliação da Atividade,Indicadores de Avaliação dos Alunos,Adequação à estratégias ODS,MMAR com os objetivos ODS indicados
0,aexGerarPDF_AEX-FMRP-00020.01,2,3223241,Regina Yoneko Dakuzaku Carretta,Caminhada da Limpeza - FMRP,FMRP-00020. 01,Faculdade de Medicina de Ribeirão Preto,FMRP,Será realizada uma caminhada no entorno do Pré...,"Comunidade FMRP - discentes, docentes, servido...",6:00,6:00,Conscientizar a comunidade sobre a importância...,Número de participantes; fotos dos ambientes a...,Frequencia e interação com as pessoas.,"Cidades e Comunidades Sustentáveis, Consumo e ...",ODS 3 - Saúde e Bem-estar: Conexão: A limpeza ...
1,aexGerarPDF_AEX-ICMC-00032.01,2,4831147,Eduardo do Valle Simões,Caminhada Solidária,ICMC-00032. 01,Instituto de Ciências Matemáticas e de Computação,ICMC,A atividade Caminhada Solidária consiste em um...,Instituições de caridade de São Carlos,20:00,5:00,A Caminhada Solidária tem como objetivo arreca...,A avaliação pelo grupo social alvo será feita ...,A avaliação dos alunos será baseada em critéri...,"Erradicação da Pobreza, Fome Zero e Agricultur...",1 - Erradicação da pobreza: A atividade contri...
2,aexGerarPDF_AEX-FZEA-00060.01,2,5412221,Caio Filipe da Motta Lima,Caminhos da Natureza: Valorizando a biodiversi...,FZEA-00060. 01,Faculdade de Zootecnia e Engenharia de Alimentos,FZEA,1 - Descrição da atividade e objetivos: Projet...,Comunidade interna: alunos de graduação e pós-...,100:00,100:00,"Objetivos, metas e resultados esperados para a...",Quantidade de participantes internos e externo...,"Presença nas reuniões, pró-atividade, entrega ...","Saúde e Bem-Estar, Educação de Qualidade, Vida...",O projeto irá utilizar de eventos e atividades...
3,aexGerarPDF_AEX-FOB-00004.01,6,1851311,Regina Tangerino de Souza Jacob,CAMP CHAT: sempre conectados - uma aventura au...,FOB-00004. 01,Faculdade de Odontologia de Bauru,FOB,O camp CHAT é um acampamento day-use focado em...,Crianças e adolescentes com deficiência auditi...,16:00,16:00,Objetivo geral:Estimular e ampliar o uso do SM...,Após as reuniões serão utilizados questionário...,Os estudantes irão relatar suas experiências e...,"Saúde e Bem-Estar, Educação de Qualidade, Redu...",A metodologia para alcançar o objetivo de Saúd...
4,aexGerarPDF_AEX-FOB-00022.01,4,1851311,Regina Tangerino de Souza Jacob,CAMP CHAT: sempre conectados - uma aventura au...,FOB-00022. 01,Faculdade de Odontologia de Bauru,FOB,O camp CHAT é um acampamento day-use focado em...,Crianças e adolescentes com deficiência auditi...,60:00,60:00,Objetivo geral: Estimular e ampliar o uso do S...,A avaliação pela comunidade será feita de form...,Os estudantes serão avaliados de forma contínu...,"Saúde e Bem-Estar, Educação de Qualidade, Redu...",A proposta se alinha a três Objetivos de Desen...


In [ ]:
df.to_csv("AEX_estruturados.csv")